# Enable model to read my goodreads data

## Imports

In [2]:
import pandas as pd

## Load Goodreads Profile from Library Download

In [3]:
profile = pd.read_csv("../raw_data/goodreads_library_export.csv")
profile.head(3)

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,...,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
0,40961543,The Reluctant Fundamentalist,Mohsin Hamid,"Hamid, Mohsin",NaN,"=""""","=""""",4,3.73,Mariner Books; 1 edition,...,2025/11/13,2025/11/16,NaN,NaN,read,What a poignant read this was. There's a push ...,NaN,NaN,1,0
1,28921,The Remains of the Day,Kazuo Ishiguro,"Ishiguro, Kazuo",NaN,"=""""","=""""",0,4.14,Faber & Faber,...,NaN,2025/10/14,to-read,to-read (#57),to-read,NaN,NaN,NaN,0,0
2,60726415,Don't Believe Everything You Think,Joseph Nguyen,"Nguyen, Joseph",NaN,"=""""","=""""",0,3.79,NaN,...,NaN,2025/07/04,to-read,to-read (#56),to-read,NaN,NaN,NaN,0,0


In [33]:
# clean columns
profile.columns = ["_".join(col.lower().strip().split()) for col in  profile.columns]
profile.columns
profile[["book_id",
         "isbn",
         "title",
         "my_rating",
         "exclusive_shelf",
         "date_read",
         "my_review"]].sample(2)


,book_id,isbn,title,my_rating,exclusive_shelf,date_read,my_review
43,8489,"=""0060598972""",Muhammad: A Prophet for Our Time (Eminent Lives),0,read,NaN,NaN
39,18143977,"=""1476746583""",All the Light We Cannot See,0,to-read,NaN,NaN


## Load Book Titles

In [6]:
book_titles = pd.read_csv("../raw_data/book_titles.csv")

## Match 
> If data is from Goodreads, match using book ids

In [32]:
# Trial match using book ids
my_book_ids = profile["book_id"]
book_titles[book_titles["book_id"].isin(my_book_ids)].shape

(103, 6)

# Check work ids and book ids in whole dataset

In [24]:
book_titles = pd.read_csv("../raw_data/book_titles.csv")

In [28]:
book_titles[["work_id", "book_id"]].isna().sum()

work_id    524
book_id      0
dtype: int64